In [1]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

In [2]:
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, wait, worker_client
from vani.core.analysis import Analysis
from vani.core.metrics import filter_asymptote_delayed, filter_delayed, flatten_delayed, merge_delayed, sort_delayed
from vani.utils.dask_agg import nunique
from vani.utils.file_utils import ensure_dir
from vani.utils.json_encoders import NpEncoder
from vani.utils.logger import create_logger, format_log


In [3]:
client = Client(LocalCluster(n_workers=16, local_directory=f"/var/tmp/playground-quartz/local"))
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:43923/status,
Dashboard: http://127.0.0.1:43923/status,Workers: 16
Total threads: 48,Total memory: 125.50 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41822,Workers: 16
Dashboard: http://127.0.0.1:43923/status,Total threads: 48
Started: Just now,Total memory: 125.50 GiB
Comm: tcp://127.0.0.1:41086,Total threads: 3
Dashboard: http://127.0.0.1:43413/status,Memory: 7.84 GiB
Nanny: tcp://127.0.0.1:43439,


In [4]:
CACHE_DIR = "cached"
METRICS_DIR = "metrics"
INDEX_DIR = "indexed"
XFER_SIZE_BINS = [
    -np.inf,
    4 * 1024.0,
    16 * 1024.0,
    64 * 1024.0,
    256 * 1024.0,
    1 * 1024.0 * 1024.0,
    4 * 1024.0 * 1024.0,
    16 * 1024.0 * 1024.0,
    64 * 1024.0 * 1024.0,
    np.inf
]
XFER_SIZE_BIN_LABELS = [
    '<4KB',
    '~16KB',
    '~64KB',
    '~256KB',
    '~1MB',
    '~4MB',
    '~16MB',
    '~64MB',
    '>64MB'
]
XFER_SIZE_BIN_NAMES = [
    '<4KB',
    '4KB',
    '16KB',
    '64KB',
    '256KB',
    '1MB',
    '4MB',
    '16MB',
    '64MB',
    '>64MB'
]


def compute_metrics_file_id(ddf: DataFrame, fg_index: str, log_dir: str):
    unique_filenames_d = unique_filenames_delayed(ddf=ddf, log_dir=log_dir,
                                                  dask_key_name=f"unique-filenames-{fg_index}")
    save_filenames_d = save_filenames_delayed(filenames=unique_filenames_d, log_dir=log_dir,
                                              dask_key_name=f"save-filenames-{fg_index}")
    metrics_d = metrics_filenames_delayed(ddf=ddf, filenames=unique_filenames_d,
                                          dask_key_name=f"metrics-{fg_index}")
    return [unique_filenames_d, save_filenames_d, [metrics_d]]


def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max


@delayed
def metrics_filenames_delayed(ddf: DataFrame, filenames: list, fg_index='file_id'):
    print('calculating metrics for filenames', len(filenames))
    tasks_d = []
    for filename in filenames:
        target_ddf_d = Analysis.target_ddf_delayed(ddf=ddf, start=filename, stop=filename,
                                                   dask_key_name=f"target-ddf-{fg_index}-{filename}")
        filter_d = filter_delayed(ddf=target_ddf_d, fg_index=fg_index, start=filename, stop=filename,
                                  dask_key_name=f"filter-{fg_index}-{filename}")
        flatten_d = flatten_delayed(filter_d, dask_key_name=f"flatten-{fg_index}-{filename}")
        tasks_d.append(flatten_d)
    print('num of tasks created', len(tasks_d))
    # with worker_client() as client:
    #     print('submitting on', client)
    #     metrics_f = client.compute(tasks_d)
    #     print('gathering tasks', len(metrics_f))
    #     metrics = client.gather(metrics_f)
    metrics = dask.compute(tasks_d)
    print('computed metrics', type(metrics))
    print('computed metrics', len(metrics))
    # print('computed metrics', metrics)
    return metrics


def read_and_index_logs(prefix: str, fg_index: str, log_dir: str, use_cache=True):
    dask_suffix = f"{prefix}-{fg_index}"
    fg_index_dir = f"{log_dir}/{INDEX_DIR}/{fg_index}"
    if use_cache and os.path.exists(f"{fg_index_dir}/_metadata"):
        ddf_d = read_parquet_delayed(log_dir=fg_index_dir, index=[fg_index],
                                     dask_key_name=f"read-parquet-{dask_suffix}")
        persisted_ddf_d = persist_ddf_delayed(ddf=ddf_d, dask_key_name=f"persist-ddf-{dask_suffix}")
        return [ddf_d, persisted_ddf_d]
    ddf_d = read_parquet_delayed(log_dir=log_dir, dask_key_name=f"read-parquet-{dask_suffix}")
    indexed_ddf_d = set_ddf_index_delayed(ddf=ddf_d, fg_index=fg_index,
                                          dask_key_name=f"set-index-{dask_suffix}")
    persisted_ddf_d = persist_ddf_delayed(ddf=indexed_ddf_d, dask_key_name=f"persist-ddf-{dask_suffix}")
    partitioned_ddf_d = repartition_delayed(ddf=persisted_ddf_d,
                                            dask_key_name=f"repartition-ddf-{dask_suffix}")
    save_ddf_d = save_ddf_delayed(ddf=partitioned_ddf_d, log_dir=log_dir, fg_index=fg_index,
                                  dask_key_name=f"save-ddf-{dask_suffix}")
    return [ddf_d, indexed_ddf_d, persisted_ddf_d, save_ddf_d, partitioned_ddf_d]


@delayed
def read_parquet_delayed(log_dir: str, index: list = None):
    if index:
        print("Index specified", index)
        return dd.read_parquet(f"{log_dir}/*.parquet", calculate_divisions=True, index=index)
    print("Index not specified")
    return dd.read_parquet(f"{log_dir}/*.parquet", index=False)


@delayed
def repartition_delayed(ddf: DataFrame, partition_size='128MB'):
    return ddf.repartition(partition_size=partition_size)


@delayed
def persist_ddf_delayed(ddf: DataFrame):
    ddf = ddf.persist()
    wait(ddf)
    return ddf


@delayed
def save_ddf_delayed(ddf: DataFrame, log_dir: str, fg_index: str):
    ddf.to_parquet(f"{log_dir}/{INDEX_DIR}/{fg_index}")


@delayed
def save_filenames_delayed(filenames: list, log_dir: str):
    filenames = list(filenames)
    filenames.sort()
    with open(f"{log_dir}/filenames.json", "w") as file:
        json.dump(filenames, file, cls=NpEncoder)


@delayed
def set_ddf_index_delayed(ddf: DataFrame, fg_index: str):
    return ddf.set_index([fg_index])


@delayed
def unique_filenames_delayed(ddf: DataFrame, log_dir: str):
    if os.path.exists(f"{log_dir}/filenames.json"):
        with open(f"{log_dir}/filenames.json", "r") as file:
            unique_filenames = json.load(file)
    else:
        unique_filenames = ddf.index.unique().compute()
    return unique_filenames


In [5]:
%%time
log_dir = "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
ddf

CPU times: user 37.6 ms, sys: 25.1 ms, total: 62.6 ms
Wall time: 103 ms


,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=10,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
%%time
stat_agg = ddf \
    .groupby(['io_cat']) \
    .agg({
        'acc_pat': [min, max],
        'app': [nunique()],
        'duration': [sum],
        'file_id': [nunique()],
        'hostname': [nunique()],
        'index': ['count'],
        'proc_id': [nunique()],
        'rank': [nunique()],
        'size': [min, max, 'mean', sum],
    }) \
    .compute()
stat_agg = stat_agg[stat_agg.index.isin([1, 2, 3])]
stat_agg['duration', 'per'] = stat_agg.div(stat_agg['duration', 'sum'].sum(), level=0)['duration', 'sum']
stat_agg['file_id', 'per'] = stat_agg.div(stat_agg['file_id', 'nunique'].max(), level=0)['file_id', 'nunique']
stat_agg['index', 'per'] = stat_agg.div(stat_agg['index', 'count'].sum(), level=0)['index', 'count']
stat_agg['proc_id', 'per'] = stat_agg.div(stat_agg['proc_id', 'nunique'].max(), level=0)['proc_id', 'nunique']
stat_agg['size', 'per'] = stat_agg.div(stat_agg['size', 'sum'].sum(), level=0)['size', 'sum']
stat_agg['bw', 'sum'] = stat_agg['size', 'sum'] / stat_agg['duration', 'sum']
stat_agg['bw', 'per'] = stat_agg.div(stat_agg['bw', 'sum'].sum(), level=0)['bw', 'sum']
stat_agg['xfer', 'max'] = pd.cut(stat_agg['size', 'max'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
stat_agg['xfer', 'min'] = pd.cut(stat_agg['size', 'min'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
stat_agg['xfer', 'mean'] = pd.cut(stat_agg['size', 'mean'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
stat_agg['bw', 'sum'] = stat_agg['bw', 'sum'].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".2f") + "GB/s")
stat_agg['size', 'sum'] = stat_agg['size', 'sum'].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".2f") + "GB")
stat_agg.drop(columns=[('size', 'max'), ('size', 'min'), ('size', 'mean')], inplace=True)
stat_agg = stat_agg.reindex(sorted(stat_agg.columns), axis=1)
stat_agg


CPU times: user 1.62 s, sys: 584 ms, total: 2.2 s
Wall time: 7.4 s


acc_pat         app        bw            duration            file_id  \
           max min nunique       per       sum       per        sum nunique   
io_cat                                                                        
1            0   0       4  0.636355  1.20GB/s  0.671074  23.869251    1597   
2            0   0       5  0.363602  0.69GB/s  0.268814   9.561380    1187   
3            1   0       5  0.000043  0.00GB/s  0.060111   2.138075    1923   

                 hostname    index           proc_id              rank  \
             per  nunique    count       per nunique       per nunique   
io_cat                                                                   
1       0.830473       32  3233729  0.647086     229  0.898039      13   
2       0.617265       32  1703563  0.340892     255  1.000000      13   
3       1.000000       32    60080  0.012022     255  1.000000      13   

            size             xfer               
             per      sum     max   mean   min  
io_cat                                          
1       0.813745  28.73GB  ~256KB  ~16KB  <4KB  
2       0.186250   6.58GB   ~16MB  ~16KB  <4KB  
3       0.000005   0.00GB    <4KB   <4KB  <4KB

In [7]:
stat_agg_desc = stat_agg.describe(percentiles=[.9])
stat_agg_desc

acc_pat            app        bw  duration                 file_id  \
            max  min   nunique       per       per        sum      nunique   
count  3.000000  3.0  3.000000  3.000000  3.000000   3.000000     3.000000   
mean   0.333333  0.0  4.666667  0.333333  0.333333  11.856236  1569.000000   
std    0.577350  0.0  0.577350  0.319234  0.310550  11.045849   368.798048   
min    0.000000  0.0  4.000000  0.000043  0.060111   2.138075  1187.000000   
50%    0.000000  0.0  5.000000  0.363602  0.268814   9.561380  1597.000000   
90%    0.800000  0.0  5.000000  0.581805  0.590622  21.007677  1857.800000   
max    1.000000  0.0  5.000000  0.636355  0.671074  23.869251  1923.000000   

                hostname         index               proc_id            \
            per  nunique         count       per     nunique       per   
count  3.000000      3.0  3.000000e+00  3.000000    3.000000  3.000000   
mean   0.815913     32.0  1.665791e+06  0.333333  246.333333  0.966013   
std    0.191783      0.0  1.587162e+06  0.317599   15.011107  0.058867   
min    0.617265     32.0  6.008000e+04  0.012022  229.000000  0.898039   
50%    0.830473     32.0  1.703563e+06  0.340892  255.000000  1.000000   
90%    0.966095     32.0  2.927696e+06  0.585847  255.000000  1.000000   
max    1.000000     32.0  3.233729e+06  0.647086  255.000000  1.000000   

         rank      size  
      nunique       per  
count     3.0  3.000000  
mean     13.0  0.333333  
std       0.0  0.426343  
min      13.0  0.000005  
50%      13.0  0.186250  
90%      13.0  0.688246  
max      13.0  0.813745

In [8]:
%%time
xfer_agg = ddf[ddf['io_cat'].isin([1, 2])] \
    .map_partitions(lambda df: df.assign(xfer=pd.cut(df['size'], XFER_SIZE_BINS, right=True))) \
    .groupby(['io_cat', 'xfer']) \
    .agg({'duration': [sum], 'size': [sum], 'index': ['count']}) \
    .compute() \
    .replace(0, np.nan) \
    .dropna()
xfer_agg


CPU times: user 896 ms, sys: 226 ms, total: 1.12 s
Wall time: 5.17 s


duration          size      index
                                      sum           sum      count
io_cat xfer                                                       
1      (-inf, 4096.0]           10.362041  6.563708e+09  2286478.0
       (4096.0, 16384.0]         5.968031  6.401671e+09   570366.0
       (16384.0, 65536.0]        5.744478  8.420302e+09   321988.0
       (65536.0, 262144.0]       1.794699  9.461608e+09    54897.0
2      (-inf, 4096.0]            4.426517  4.563062e+09  1584932.0
       (4096.0, 16384.0]         2.960737  8.093822e+08    90644.0
       (16384.0, 65536.0]        1.221840  3.839877e+08    16450.0
       (65536.0, 262144.0]       0.937759  1.186261e+09    11508.0
       (1048576.0, 4194304.0]    0.007893  6.446750e+07       17.0
       (4194304.0, 16777216.0]   0.006635  5.316648e+07       12.0

In [9]:
def format_agg(df, add_xfer=True):
    df['bw', 'sum'] = df['size', 'sum'] / df['duration', 'sum']
    df['bw', 'per'] = df.div(df['bw', 'sum'].sum(), level=0)['bw', 'sum']
    df['duration', 'per'] = df.div(df['duration', 'sum'].sum(), level=0)['duration', 'sum']
    df['index', 'per'] = df.div(df['index', 'count'].sum(), level=0)['index', 'count']
    df['size', 'per'] = df.div(df['size', 'sum'].sum(), level=0)['size', 'sum']
    if add_xfer:
        df['xfer', 'max'] = pd.cut(df['size', 'max'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
        df['xfer', 'min'] = pd.cut(df['size', 'min'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
        df['xfer', 'mean'] = pd.cut(df['size', 'mean'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
        df.drop(columns=[('size', 'max'), ('size', 'min'), ('size', 'mean')], inplace=True)
    df['bw', 'fmt'] = df['bw', 'sum'].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".2f") + "GB/s")
    df['size', 'fmt'] = df['size', 'sum'].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".2f") + "GB")
    df = df.reindex(sorted(df.columns), axis=1)
    return df

def filter_xfer_agg(xfer_agg, io_cat):
    xfer_agg_cat = xfer_agg.loc[(io_cat,)]
    xfer_agg_cat = format_agg(xfer_agg_cat, add_xfer=False)
    return xfer_agg_cat

xfer_agg_read = filter_xfer_agg(xfer_agg, io_cat=1)
xfer_agg_read

bw                          duration             \
                          fmt       per           sum       per        sum   
xfer                                                                         
(-inf, 4096.0]       0.59GB/s  0.075017  6.334377e+08  0.434117  10.362041   
(4096.0, 16384.0]    1.00GB/s  0.127034  1.072660e+09  0.250030   5.968031   
(16384.0, 65536.0]   1.37GB/s  0.173594  1.465808e+09  0.240664   5.744478   
(65536.0, 262144.0]  4.91GB/s  0.624354  5.271975e+09  0.075189   1.794699   

                         index              size                          
                         count       per     fmt       per           sum  
xfer                                                                      
(-inf, 4096.0]       2286478.0  0.707072  6.11GB  0.212781  6.563708e+09  
(4096.0, 16384.0]     570366.0  0.176380  5.96GB  0.207528  6.401671e+09  
(16384.0, 65536.0]    321988.0  0.099572  7.84GB  0.272967  8.420302e+09  
(65536.0, 262144.0]    54897.0  0.016976  8.81GB  0.306724  9.461608e+09

In [10]:
xfer_agg_write = filter_xfer_agg(xfer_agg, io_cat=2)
xfer_agg_write

bw                          duration            \
                              fmt       per           sum       per       sum   
xfer                                                                            
(-inf, 4096.0]           0.96GB/s  0.054073  1.030847e+09  0.462958  4.426517   
(4096.0, 16384.0]        0.25GB/s  0.014340  2.733719e+08  0.309656  2.960737   
(16384.0, 65536.0]       0.29GB/s  0.016485  3.142700e+08  0.127789  1.221840   
(65536.0, 262144.0]      1.18GB/s  0.066355  1.264997e+09  0.098078  0.937759   
(1048576.0, 4194304.0]   7.61GB/s  0.428407  8.167163e+09  0.000826  0.007893   
(4194304.0, 16777216.0]  7.46GB/s  0.420341  8.013395e+09  0.000694  0.006635   

                             index              size                          
                             count       per     fmt       per           sum  
xfer                                                                          
(-inf, 4096.0]           1584932.0  0.930363  4.25GB  0.646296  4.563062e+09  
(4096.0, 16384.0]          90644.0  0.053208  0.75GB  0.114638  8.093822e+08  
(16384.0, 65536.0]         16450.0  0.009656  0.36GB  0.054387  3.839877e+08  
(65536.0, 262144.0]        11508.0  0.006755  1.10GB  0.168018  1.186261e+09  
(1048576.0, 4194304.0]        17.0  0.000010  0.06GB  0.009131  6.446750e+07  
(4194304.0, 16777216.0]       12.0  0.000007  0.05GB  0.007530  5.316648e+07

In [11]:
PERCENTILE = .95
PERCENTILE_FMT = f"{int(PERCENTILE * 100)}%"


In [12]:
%%time
tmin_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
tmin_min, tmid_max

CPU times: user 1.06 ms, sys: 1.04 ms, total: 2.1 ms
Wall time: 2.49 ms


(0, 1752590553)

In [13]:
%%time
num_time_tasks = 2 ** 10
time_interval = math.ceil(tmid_max / num_time_tasks)
# time_interval = math.ceil(time_max / 60 / 60 / 60)
trange = np.arange(0, tmid_max, time_interval)
trange

CPU times: user 42 µs, sys: 23 µs, total: 65 µs
Wall time: 73.2 µs


array([         0,    1711515,    3423030, ..., 1747456815, 1749168330,
       1750879845])

In [14]:
%%time
ddf = ddf.map_partitions(lambda df: df.assign(trange=pd.cut(df['tmid'], trange, right=True)))

CPU times: user 53.3 ms, sys: 5.62 ms, total: 58.9 ms
Wall time: 51.9 ms


In [15]:
%%time
trange_agg = ddf \
    .groupby(['trange','io_cat']) \
    .agg({'duration':sum,'size':sum,'index':'count'}) \
    .compute()
trange_agg.sort_values('duration', ascending=False)  

CPU times: user 880 ms, sys: 183 ms, total: 1.06 s
Wall time: 4.77 s


duration  size  index
trange                   io_cat                        
(10269090, 11980605]     0       12.366123     0     28
(8557575, 10269090]      0        7.690526     0     28
(6846060, 8557575]       0        7.492290     0     32
(20538180, 22249695]     0        5.910713     0      4
(5134545, 6846060]       0        2.992074     0     14
...                                    ...   ...    ...
(727393875, 729105390]   1        0.000000     0      0
                         2        0.000000     0      0
                         3        0.000000     0      0
(729105390, 730816905]   0        0.000000     0      0
(1749168330, 1750879845] 3        0.000000     0      0

[4092 rows x 3 columns]

In [16]:
trange_agg_sum = trange_agg.groupby(level=0).sum().sort_values('duration', ascending=False)
trange_agg_sum

,duration,size,index
trange,,,
"(10269090, 11980605]",13.968435,1623116096,263661
"(6846060, 8557575]",10.188845,1929305172,323897
"(8557575, 10269090]",9.704213,1790802648,280793
"(5134545, 6846060]",7.177463,2390054771,395098
"(20538180, 22249695]",6.088249,316794869,15397
...,...,...,...
"(871161135, 872872650]",0.000000,0,0
"(872872650, 874584165]",0.000000,0,0
"(876295680, 878007195]",0.000000,0,0


In [17]:
trange_agg_sum_desc = trange_agg_sum.describe(percentiles=[PERCENTILE])
trange_agg_sum_desc

,duration,size,index
count,1023.000000,1.023000e+03,1023.000000
mean,0.079491,3.705207e+07,4885.458456
std,0.725695,1.692446e+08,26618.442476
min,0.000000,0.000000e+00,0.000000
50%,0.000000,0.000000e+00,0.000000
95%,0.118704,1.831188e+08,24435.800000
max,13.968435,2.390055e+09,395098.000000


In [18]:
trange_agg_sum[trange_agg_sum['duration'] > 0].count()

duration    251
size        251
index       251
dtype: int64

In [19]:
trange_agg_sum[trange_agg_sum['duration'] > trange_agg_sum_desc.loc[PERCENTILE_FMT]['duration']].count()

duration    52
size        52
index       52
dtype: int64

In [20]:
trange_agg_cut = trange_agg_sum[trange_agg_sum['duration'] > trange_agg_sum_desc.loc[PERCENTILE_FMT]['duration']].sort_values('duration', ascending=False)
trange_agg_cut

,duration,size,index
trange,,,
"(10269090, 11980605]",13.968435,1623116096,263661
"(6846060, 8557575]",10.188845,1929305172,323897
"(8557575, 10269090]",9.704213,1790802648,280793
"(5134545, 6846060]",7.177463,2390054771,395098
"(20538180, 22249695]",6.088249,316794869,15397
"(1711515, 3423030]",4.183501,932854362,147469
"(3423030, 5134545]",3.476630,1934782053,317897
"(13692120, 15403635]",3.341134,1302817322,182995
"(18826665, 20538180]",3.069379,359696625,13653


In [21]:
%%time
trange_agg_file = ddf[ddf['trange'].isin(trange_agg_cut.index)] \
    .groupby(['file_id','io_cat']) \
    .agg({
        'acc_pat': [min, max],
        'app': [nunique()],
        'duration': [sum],
#         'file_id': [nunique()],
        'hostname': [nunique()],
        'index': ['count'],
        'proc_id': [nunique()],
        'rank': [nunique()],
        'size': [min, max, 'mean', sum],
    }) \
    .compute()
trange_agg_file = trange_agg_file.sort_values(('duration', 'sum'), ascending=False)
trange_agg_file = format_agg(trange_agg_file)
trange_agg_file

CPU times: user 1.09 s, sys: 239 ms, total: 1.33 s
Wall time: 5.97 s


acc_pat         app        bw            \
                               max min nunique       fmt       per   
file_id             io_cat                                           
6142509188972423790 0            0   0       1  0.00GB/s  0.000000   
7701937538982921416 1            0   0       1  0.59GB/s  0.000225   
7668160541777642696 1            0   0       1  0.70GB/s  0.000266   
7639450094153155784 1            0   0       2  4.45GB/s  0.001695   
7707004088563713224 1            0   0       1  0.77GB/s  0.000294   
...                            ...  ..     ...       ...       ...   
7674352990043551940 2            0   0       1  0.01GB/s  0.000004   
7642827792651958468 2            0   0       1  0.01GB/s  0.000004   
7683641664275003588 2            0   0       1  0.01GB/s  0.000004   
7686174939065399492 2            0   0       1  0.01GB/s  0.000004   
7703344912644749508 2            0   0       1  0.01GB/s  0.000004   

                                              duration            hostname  \
                                     sum           per        sum  nunique   
file_id             io_cat                                                   
6142509188972423790 0       0.000000e+00  5.949928e-01  45.752838       28   
7701937538982921416 1       6.347715e+08  3.912663e-02   3.008699        1   
7668160541777642696 1       7.495438e+08  1.803351e-02   1.386713        2   
7639450094153155784 1       4.783079e+09  1.288887e-02   0.991109        2   
7707004088563713224 1       8.306249e+08  1.180085e-02   0.907444        1   
...                                  ...           ...        ...      ...   
7674352990043551940 2       1.217391e+07  8.973105e-08   0.000007        1   
7642827792651958468 2       1.217391e+07  8.973105e-08   0.000007        1   
7683641664275003588 2       1.217391e+07  8.973105e-08   0.000007        1   
7686174939065399492 2       1.217391e+07  8.973105e-08   0.000007        1   
7703344912644749508 2       1.217391e+07  8.973105e-08   0.000007        1   

                             index           proc_id    rank    size  \
                             count       per nunique nunique     fmt   
file_id             io_cat                                             
6142509188972423790 0          456  0.000108     114      13  0.00GB   
7701937538982921416 1       103574  0.024478       1       1  1.78GB   
7668160541777642696 1       169718  0.040110       2       1  0.97GB   
7639450094153155784 1        27510  0.006501       2       1  4.41GB   
7707004088563713224 1        53385  0.012617       1       1  0.70GB   
...                            ...       ...     ...     ...     ...   
7674352990043551940 2            6  0.000001       1       1  0.00GB   
7642827792651958468 2            6  0.000001       1       1  0.00GB   
7683641664275003588 2            6  0.000001       1       1  0.00GB   
7686174939065399492 2            6  0.000001       1       1  0.00GB   
7703344912644749508 2            6  0.000001       1       1  0.00GB   

                                                        xfer                
                                     per         sum     max    mean   min  
file_id             io_cat                                                  
6142509188972423790 0       0.000000e+00           0    <4KB    <4KB  <4KB  
7701937538982921416 1       6.677920e-02  1909836564   ~64KB   ~64KB  <4KB  
7668160541777642696 1       3.634365e-02  1039402040   ~16KB   ~16KB  <4KB  
7639450094153155784 1       1.657577e-01  4740550404  ~256KB  ~256KB  <4KB  
7707004088563713224 1       2.635541e-02   753745484   ~16KB   ~16KB  <4KB  
...                                  ...         ...     ...     ...   ...  
7674352990043551940 2       2.937137e-09          84    <4KB    <4KB  <4KB  
7642827792651958468 2       2.937137e-09          84    <4KB    <4KB  <4KB  
7683641664275003588 2       2.937137e-09          84    <4KB    <4KB  <4KB  
7686174939065399492 

In [22]:
trange_agg_file_desc = trange_agg_file.describe(percentiles=[PERCENTILE])
trange_agg_file_desc

acc_pat                  app            bw                \
               max     min      nunique           per           sum   
count  4028.000000  4028.0  4028.000000  4.028000e+03  4.028000e+03   
mean      0.137786     0.0     1.500000  2.482622e-04  7.007573e+08   
std       0.344718     0.0     0.568829  4.289499e-04  1.210776e+09   
min       0.000000     0.0     1.000000 -2.053780e-08 -5.797101e+04   
50%       0.000000     0.0     1.000000  4.662220e-05  1.315982e+08   
95%       1.000000     0.0     2.000000  1.344194e-03  3.794190e+09   
max       1.000000     0.0     3.000000  3.159107e-03  8.917054e+09   

           duration                  hostname          index                \
                per          sum      nunique          count           per   
count  4.028000e+03  4028.000000  4028.000000    4028.000000  4.028000e+03   
mean   2.482622e-04     0.019090     1.107001    1050.485601  2.482622e-04   
std    9.419443e-03     0.724322     0.808276    9537.032803  2.253895e-03   
min    8.973105e-08     0.000007     1.000000       1.000000  2.363309e-07   
50%    5.909895e-06     0.000454     1.000000      12.000000  2.835970e-06   
95%    9.132814e-05     0.007023     2.000000    1521.650000  3.596129e-04   
max    5.949928e-01    45.752838    31.000000  190486.000000  4.501772e-02   

           proc_id         rank          size                
           nunique      nunique           per           sum  
count  4028.000000  4028.000000  4.028000e+03  4.028000e+03  
mean      1.779791     1.287239  2.482622e-04  7.100118e+06  
std       2.278568     0.992230  3.300194e-03  9.438316e+07  
min       1.000000     1.000000 -5.594548e-10 -1.600000e+01  
50%       1.000000     1.000000  1.007019e-07  2.880000e+03  
95%       4.000000     4.000000  2.139470e-04  6.118728e+06  
max     114.000000    13.000000  1.657577e-01  4.740550e+09

In [23]:
trange_agg_file_cut = trange_agg_file[trange_agg_file[('duration', 'sum')] > trange_agg_file_desc.loc[PERCENTILE_FMT]['duration']['sum']]
trange_agg_file_cut = trange_agg_file_cut.sort_values(('duration', 'per'), ascending=False)
trange_agg_file_cut

acc_pat         app        bw                \
                               max min nunique       fmt           per   
file_id             io_cat                                               
6142509188972423790 0            0   0       1  0.00GB/s  0.000000e+00   
7701937538982921416 1            0   0       1  0.59GB/s  2.248849e-04   
7668160541777642696 1            0   0       1  0.70GB/s  2.655461e-04   
7639450094153155784 1            0   0       2  4.45GB/s  1.694535e-03   
7707004088563713224 1            0   0       1  0.77GB/s  2.942713e-04   
...                            ...  ..     ...       ...           ...   
7642827793873683656 2            0   0       1  0.39GB/s  1.483520e-04   
4335284040774263594 1            0   0       3  0.31GB/s  1.162786e-04   
4983802387181081216 2            0   0       1  0.97GB/s  3.691850e-04   
7646768441647617231 3            1   0       2  0.00GB/s  4.022955e-07   
7361702989253236945 1            0   0       2  0.03GB/s  9.554044e-06   

                                          duration            hostname  \
                                     sum       per        sum  nunique   
file_id             io_cat                                               
6142509188972423790 0       0.000000e+00  0.594993  45.752838       28   
7701937538982921416 1       6.347715e+08  0.039127   3.008699        1   
7668160541777642696 1       7.495438e+08  0.018034   1.386713        2   
7639450094153155784 1       4.783079e+09  0.012889   0.991109        2   
7707004088563713224 1       8.306249e+08  0.011801   0.907444        1   
...                                  ...       ...        ...      ...   
7642827793873683656 2       4.187458e+08  0.000097   0.007456        1   
4335284040774263594 1       3.282138e+08  0.000096   0.007345        2   
4983802387181081216 2       1.042080e+09  0.000093   0.007180        1   
7646768441647617231 3       1.135539e+06  0.000092   0.007059        2   
7361702989253236945 1       2.696772e+07  0.000092   0.007048        2   

                             index           proc_id    rank    size  \
                             count       per nunique nunique     fmt   
file_id             io_cat                                             
6142509188972423790 0          456  0.000108     114      13  0.00GB   
7701937538982921416 1       103574  0.024478       1       1  1.78GB   
7668160541777642696 1       169718  0.040110       2       1  0.97GB   
7639450094153155784 1        27510  0.006501       2       1  4.41GB   
7707004088563713224 1        53385  0.012617       1       1  0.70GB   
...                            ...       ...     ...     ...     ...   
7642827793873683656 2          431  0.000102       1       1  0.00GB   
4335284040774263594 1          840  0.000199       4       3  0.00GB   
4983802387181081216 2         2598  0.000614       1       1  0.01GB   
7646768441647617231 3          520  0.000123       9       9  0.00GB   
7361702989253236945 1           67  0.000016       2       2  0.00GB   

                                                        xfer                
                                     per         sum     max    mean   min  
file_id             io_cat                                                  
6142509188972423790 0       0.000000e+00           0    <4KB    <4KB  <4KB  
7701937538982921416 1       6.677920e-02  1909836564   ~64KB   ~64KB  <4KB  
7668160541777642696 1       3.634365e-02  1039402040   ~16KB   ~16KB  <4KB  
7639450094153155784 1       1.657577e-01  4740550404  ~256KB  ~256KB  <4KB  
7707004088563713224 1       2.635541e-02   753745484   ~16KB   ~16KB  <4KB  
...                                  ...         ...     ...     ...   ...  
7642827793873683656 2       1.091636e-04     3122001   ~16KB   ~16KB  <4KB  
4335284040774263594 1       8.428766e-05     2410566    <4KB    <4KB  <4KB  
4983802387181081216 2       2.616234e-04     7482240    <4KB    <4KB  <4KB  
7646768441647617231 

In [24]:
%%time
trange_agg_proc = ddf[ddf['trange'].isin(trange_agg_cut.index)] \
    .groupby(['proc_id','io_cat']) \
    .agg({
        'acc_pat': [min, max],
        'app': [nunique()],
        'duration': [sum],
        'file_id': [nunique()],
        'hostname': [nunique()],
        'index': ['count'],
        # 'proc_id': [nunique()],
        'rank': [nunique()],
        'size': [min, max, 'mean', sum],
    }) \
    .compute()
trange_agg_proc = trange_agg_proc.sort_values(('duration', 'sum'), ascending=False)
trange_agg_proc = format_agg(trange_agg_proc)
trange_agg_proc

CPU times: user 1.11 s, sys: 253 ms, total: 1.37 s
Wall time: 6.15 s


acc_pat         app        bw            \
                               max min nunique       fmt       per   
proc_id             io_cat                                           
9060729382066249156 1            0   0       1  0.59GB/s  0.001605   
5670209836996417988 0            0   0       1  0.00GB/s  0.000000   
5670209836996483524 0            0   0       1  0.00GB/s  0.000000   
5670209836996614596 0            0   0       1  0.00GB/s  0.000000   
5670209836996549060 0            0   0       1  0.00GB/s  0.000000   
...                            ...  ..     ...       ...       ...   
6361742405359684916 2            0   0       1  0.01GB/s  0.000031   
5841728651585965364 2            0   0       1  0.01GB/s  0.000031   
2463789624070294836 2            0   0       1  0.01GB/s  0.000031   
6347012930982169908 2            0   0       1  0.01GB/s  0.000031   
5670209343073602868 2            0   0       1  0.01GB/s  0.000031   

                                              duration           file_id  \
                                     sum           per       sum nunique   
proc_id             io_cat                                                 
9060729382066249156 1       6.347715e+08  3.912663e-02  3.008699       1   
5670209836996417988 0       0.000000e+00  2.429120e-02  1.867907       1   
5670209836996483524 0       0.000000e+00  2.013386e-02  1.548222       1   
5670209836996614596 0       0.000000e+00  1.987148e-02  1.528046       1   
5670209836996549060 0       0.000000e+00  1.923710e-02  1.479265       1   
...                                  ...           ...       ...     ...   
6361742405359684916 2       1.217391e+07  8.973106e-08  0.000007       1   
5841728651585965364 2       1.217391e+07  8.973106e-08  0.000007       1   
2463789624070294836 2       1.217391e+07  8.973106e-08  0.000007       1   
6347012930982169908 2       1.217391e+07  8.973106e-08  0.000007       1   
5670209343073602868 2       1.217391e+07  8.973106e-08  0.000007       1   

                           hostname   index                  rank    size  \
                            nunique   count           per nunique     fmt   
proc_id             io_cat                                                  
9060729382066249156 1             1  103574  2.447773e-02       1  1.78GB   
5670209836996417988 0             1       4  9.453234e-07       1  0.00GB   
5670209836996483524 0             1       4  9.453234e-07       1  0.00GB   
5670209836996614596 0             1       4  9.453234e-07       1  0.00GB   
5670209836996549060 0             1       4  9.453234e-07       1  0.00GB   
...                             ...     ...           ...     ...     ...   
6361742405359684916 2             1       6  1.417985e-06       1  0.00GB   
5841728651585965364 2             1       6  1.417985e-06       1  0.00GB   
2463789624070294836 2             1       6  1.417985e-06       1  0.00GB   
6347012930982169908 2             1       6  1.417985e-06       1  0.00GB   
5670209343073602868 2             1       6  1.417985e-06       1  0.00GB   

                                                       xfer               
                                     per         sum    max   mean   min  
proc_id             io_cat                                                
9060729382066249156 1       6.677920e-02  1909836564  ~64KB  ~64KB  <4KB  
5670209836996417988 0       0.000000e+00           0   <4KB   <4KB  <4KB  
5670209836996483524 0       0.000000e+00           0   <4KB   <4KB  <4KB  
5670209836996614596 0       0.000000e+00           0   <4KB   <4KB  <4KB  
5670209836996549060 0       0.000000e+00           0   <4KB   <4KB  <4KB  
...                                  ...         ...    ...    ...   ...  
6361742405359684916 2       2.937137e-09          84   <4KB   <4KB  <4KB  
5841728651585965364 2       2.937137e-09          84   <4KB   <4KB  <4KB  
2463789624070294836 2       2.937137e-09          84   <4KB   <4KB  <4KB  
63

In [25]:
trange_agg_proc_desc = trange_agg_proc.describe(percentiles=[PERCENTILE])
trange_agg_proc_desc

acc_pat            app            bw                    duration  \
              max    min nunique           per           sum           per   
count  828.000000  828.0   828.0  8.280000e+02  8.280000e+02  8.280000e+02   
mean     0.240338    0.0     1.0  1.207729e-03  4.776090e+08  1.207730e-03   
std      0.427547    0.0     0.0  2.718872e-03  1.075206e+09  2.942409e-03   
min      0.000000    0.0     1.0 -5.519106e-08 -2.182587e+04  8.973106e-08   
50%      0.000000    0.0     1.0  9.617340e-05  3.803276e+07  1.070973e-04   
95%      1.000000    0.0     1.0  8.760251e-03  3.464331e+09  6.667308e-03   
max      1.000000    0.0     1.0  2.254855e-02  8.917054e+09  3.912663e-02   

                      file_id hostname          index                  rank  \
              sum     nunique  nunique          count           per nunique   
count  828.000000  828.000000    828.0     828.000000  8.280000e+02   828.0   
mean     0.092870    8.658213      1.0    5110.333333  1.207729e-03     1.0   
std      0.226261   11.944970      0.0   21120.435152  4.991411e-03     0.0   
min      0.000007    1.000000      1.0       1.000000  2.363309e-07     1.0   
50%      0.008235    2.000000      1.0      36.000000  8.507911e-06     1.0   
95%      0.512692   37.000000      1.0   31794.900000  7.514116e-03     1.0   
max      3.008699   51.000000      1.0  190486.000000  4.501772e-02     1.0   

               size                
                per           sum  
count  8.280000e+02  8.280000e+02  
mean   1.207729e-03  3.454019e+07  
std    7.160216e-03  2.047770e+08  
min   -2.237819e-09 -6.400000e+01  
50%    3.503585e-08  1.002000e+03  
95%    3.874494e-03  1.108077e+08  
max    1.657576e-01  4.740548e+09

In [26]:
trange_agg_proc_cut_dur = trange_agg_proc[trange_agg_proc[('duration', 'sum')] > trange_agg_proc_desc.loc[PERCENTILE_FMT]['duration']['sum']]
trange_agg_proc_cut_dur = trange_agg_proc_cut_dur.sort_values(('duration', 'per'), ascending=False)
trange_agg_proc_cut_dur

acc_pat         app        bw            \
                               max min nunique       fmt       per   
proc_id             io_cat                                           
9060729382066249156 1            0   0       1  0.59GB/s  0.001605   
5670209836996417988 0            0   0       1  0.00GB/s  0.000000   
5670209836996483524 0            0   0       1  0.00GB/s  0.000000   
5670209836996614596 0            0   0       1  0.00GB/s  0.000000   
5670209836996549060 0            0   0       1  0.00GB/s  0.000000   
5901139297808997828 1            0   0       1  4.45GB/s  0.012095   
1029742319298993604 1            0   0       1  0.77GB/s  0.002100   
6439047951218497988 0            0   0       1  0.00GB/s  0.000000   
6439047951218563524 0            0   0       1  0.00GB/s  0.000000   
6439047951218432452 0            0   0       1  0.00GB/s  0.000000   
6439047951218629060 0            0   0       1  0.00GB/s  0.000000   
6120540898898729412 0            0   0       1  0.00GB/s  0.000000   
2463791410776694212 1            0   0       1  0.81GB/s  0.002191   
7500532995116292548 0            0   0       1  0.00GB/s  0.000000   
6120540898898532804 0            0   0       1  0.00GB/s  0.000000   
5670211164141574596 0            0   0       1  0.00GB/s  0.000000   
1701698302174617028 1            0   0       1  0.80GB/s  0.002175   
6120540898898663876 0            0   0       1  0.00GB/s  0.000000   
6120540898898598340 0            0   0       1  0.00GB/s  0.000000   
5670211164141640132 0            0   0       1  0.00GB/s  0.000000   
5670211164141705668 0            0   0       1  0.00GB/s  0.000000   
5670211164141771204 0            0   0       1  0.00GB/s  0.000000   
2439962764714761668 1            0   0       1  0.85GB/s  0.002312   
145424233979566532  0            0   0       1  0.00GB/s  0.000000   
244024996488338884  1            0   0       1  2.19GB/s  0.005936   
5012278936682092996 0            0   0       1  0.00GB/s  0.000000   
6032104343996521924 1            0   0       1  0.93GB/s  0.002520   
1705359014470147524 1            0   0       1  0.97GB/s  0.002624   
5012278936681896388 0            0   0       1  0.00GB/s  0.000000   
5012278936682027460 0            0   0       1  0.00GB/s  0.000000   
4459701488193952196 0            0   0       1  0.00GB/s  0.000000   
6120541749300485572 1            0   0       1  0.56GB/s  0.001514   
268174120124997060  1            0   0       1  0.91GB/s  0.002476   
226293052208111044  1            0   0       1  0.96GB/s  0.002618   
268173050679515588  0            0   0       1  0.00GB/s  0.000000   
2402800680445205956 1            0   0       1  0.64GB/s  0.001733   
7825775681438014916 1            0   0       1  0.97GB/s  0.002638   
145425196050865604  1            0   0       1  0.80GB/s  0.002161   
8716201128649350596 1            0   0       1  0.84GB/s  0.002280   
1458150107185732036 0            0   0       1  0.00GB/s  0.000000   
115747461661777348  1            0   0       1  1.05GB/s  0.002838   
1029741344342792644 0            0   0       1  0.00GB/s  0.000000   

                                          duration           file_id hostname  \
                                     sum       per       sum nunique  nunique   
proc_id             io_cat                                                      
9060729382066249156 1       6.347715e+08  0.039127  3.008699       1        1   
5670209836996417988 0       0.000000e+00  0.024291  1.867907       1        1   
5670209836996483524 0       0.000000e+00  0.020134  1.548222       1        1   
5670209836996614596 0       0.000000e+00  0.019871  1.528046       1        1   
5670209836996549060 0       0.000000e+00  0.019237  1.479265       1        1   
5901139297808997828 1       4.783120e+09  0.012889  0.991099       1        1   
1029742319298993604 1       8.306249e+08  0.011801  0.907444       1        1   
6439047951218497988 0       0.000000e+00  0.011457  0.881000      

In [27]:
trange_agg_proc_cut_bw = trange_agg_proc[trange_agg_proc[('bw', 'sum')] < trange_agg_proc_desc.loc['50%']['bw']['sum']]
trange_agg_proc_cut_bw = trange_agg_proc_cut_bw.sort_values(('bw', 'per'), ascending=True)
trange_agg_proc_cut_bw

acc_pat         app         bw                \
                               max min nunique        fmt           per   
proc_id             io_cat                                                
8716199904583666996 3            1   0       1  -0.00GB/s -5.519106e-08   
5012277910183136564 3            1   0       1  -0.00GB/s -5.393120e-08   
1458149123638218036 3            1   0       1  -0.00GB/s -5.153380e-08   
6032102454210907444 3            1   0       1  -0.00GB/s -5.044158e-08   
145422717854731572  3            1   0       1  -0.00GB/s -4.971791e-08   
...                            ...  ..     ...        ...           ...   
6032103223010053428 2            0   0       1   0.03GB/s  9.236029e-05   
268172153029971252  2            0   0       1   0.03GB/s  9.302001e-05   
1701695811093581108 2            0   0       1   0.03GB/s  9.302002e-05   
6347012797838183732 2            0   0       1   0.03GB/s  9.302002e-05   
6082818282740567348 2            0   0       1   0.03GB/s  9.368922e-05   

                                              duration           file_id  \
                                     sum           per       sum nunique   
proc_id             io_cat                                                 
8716199904583666996 3      -2.182587e+04  3.813310e-05  0.002932      23   
5012277910183136564 3      -2.132765e+04  3.902391e-05  0.003001      23   
1458149123638218036 3      -2.037957e+04  4.083934e-05  0.003140      23   
6032102454210907444 3      -1.994764e+04  4.172364e-05  0.003208      23   
145422717854731572  3      -1.966145e+04  4.233095e-05  0.003255      23   
...                                  ...           ...       ...     ...   
6032103223010053428 2       3.652482e+07  1.833635e-07  0.000014       1   
268172153029971252  2       3.678571e+07  1.820630e-07  0.000014       1   
1701695811093581108 2       3.678572e+07  1.820630e-07  0.000014       1   
6347012797838183732 2       3.678572e+07  1.820630e-07  0.000014       1   
6082818282740567348 2       3.705036e+07  1.807626e-07  0.000014       1   

                           hostname index              rank     size  \
                            nunique count       per nunique      fmt   
proc_id             io_cat                                             
8716199904583666996 3             1   644  0.000152       1  -0.00GB   
5012277910183136564 3             1   644  0.000152       1  -0.00GB   
1458149123638218036 3             1   644  0.000152       1  -0.00GB   
6032102454210907444 3             1   644  0.000152       1  -0.00GB   
145422717854731572  3             1   644  0.000152       1  -0.00GB   
...                             ...   ...       ...     ...      ...   
6032103223010053428 2             1     7  0.000002       1   0.00GB   
268172153029971252  2             1     7  0.000002       1   0.00GB   
1701695811093581108 2             1     7  0.000002       1   0.00GB   
6347012797838183732 2             1     7  0.000002       1   0.00GB   
6082818282740567348 2             1     7  0.000002       1   0.00GB   

                                               xfer              
                                     per  sum   max  mean   min  
proc_id             io_cat                                       
8716199904583666996 3      -2.237819e-09  -64  <4KB  <4KB  <4KB  
5012277910183136564 3      -2.237819e-09  -64  <4KB  <4KB  <4KB  
1458149123638218036 3      -2.237819e-09  -64  <4KB  <4KB  <4KB  
6032102454210907444 3      -2.237819e-09  -64  <4KB  <4KB  <4KB  
145422717854731572  3      -2.237819e-09  -64  <4KB  <4KB  <4KB  
...                                  ...  ...   ...   ...   ...  
6032103223010053428 2       1.800745e-08  515  <4KB  <4KB  <4KB  
268172153029971252  2       1.800745e-08  515  <4KB  <4KB  <4KB  
1701695811093581108 2       1.800745e-08  515  <4KB  <4KB  <4KB  
6347012797838183732 2       1.800745e-08  515  <4KB  <4KB  <4KB  
6082818282740567348 2       1.800745e-08  51

In [33]:
%%time
file_agg = ddf \
    .groupby(['file_id']) \
    .agg({
        'acc_pat': [min, max],
        'app': [nunique()],
        'duration': [sum],
        # 'file_id': [nunique()],
        'hostname': [nunique()],
        'index': ['count'],
        'io_cat': [min, max],
        'proc_id': [nunique()],
        'rank': [nunique()],
        'size': [min, max, 'mean', sum],
    }) \
    .compute()
file_agg = file_agg.sort_values(('duration', 'sum'), ascending=False)
file_agg = format_agg(file_agg)
file_agg

CPU times: user 991 ms, sys: 225 ms, total: 1.22 s
Wall time: 5.44 s


acc_pat         app        bw                          \
                        max min nunique       fmt       per           sum   
file_id                                                                     
6142509188972423790       0   0       1  0.00GB/s  0.000000  0.000000e+00   
7701937538982921416       0   0       2  0.66GB/s  0.000283  7.137414e+08   
7639450094153155784       0   0       2  4.74GB/s  0.002021  5.093748e+09   
7668160541777642696       0   0       2  0.70GB/s  0.000299  7.529625e+08   
7707004088563713224       0   0       2  0.69GB/s  0.000296  7.456980e+08   
...                     ...  ..     ...       ...       ...           ...   
8459486724935419126       0   0       1  0.00GB/s  0.000000  0.000000e+00   
8459710870688662774       0   0       1  0.00GB/s  0.000000  0.000000e+00   
1571755611344956586       0   0       1  0.00GB/s  0.000000  0.000000e+00   
1571755611836627269       0   0       1  0.00GB/s  0.000000  0.000000e+00   
1571755609301476523       0   0       1  0.00GB/s  0.000000  0.000000e+00   

                         duration            hostname   index  ... io_cat      \
                              per        sum  nunique   count  ...    max min   
file_id                                                        ...              
6142509188972423790  5.626165e-01  45.752838       28     456  ...      0   0   
7701937538982921416  4.874895e-02   3.964339        2  153464  ...      3   1   
7639450094153155784  2.303736e-02   1.873434        2   55454  ...      3   1   
7668160541777642696  1.874344e-02   1.524246        4  178024  ...      3   1   
7707004088563713224  1.547006e-02   1.258049        2   66459  ...      3   1   
...                           ...        ...      ...     ...  ...    ...  ..   
8459486724935419126  1.082124e-07   0.000009        1       1  ...      3   3   
8459710870688662774  1.082124e-07   0.000009        1       1  ...      3   3   
1571755611344956586  1.082124e-07   0.000009        1       1  ...      3   3   
1571755611836627269  1.082124e-07   0.000009        1       1  ...      3   3   
1571755609301476523  1.082124e-07   0.000009        1       1  ...      3   3   

                    proc_id    rank    size                          xfer  \
                    nunique nunique     fmt       per         sum     max   
file_id                                                                     
6142509188972423790     114      13  0.00GB  0.000000           0    <4KB   
7701937538982921416       5       5  2.64GB  0.074642  2829513124   ~64KB   
7639450094153155784       2       1  8.89GB  0.251737  9542800389  ~256KB   
7668160541777642696      10       5  1.07GB  0.030276  1147699816   ~16KB   
7707004088563713224       5       5  0.87GB  0.024748   938124444   ~16KB   
...                     ...     ...     ...       ...         ...     ...   
8459486724935419126       1       1  0.00GB  0.000000           0    <4KB   
8459710870688662774       1       1  0.00GB  0.000000           0    <4KB   
1571755611344956586       1       1  0.00GB  0.000000           0    <4KB   
1571755611836627269       1       1  0.00GB  0.000000           0    <4KB   
1571755609301476523       1       1  0.00GB  0.000000           0    <4KB   

                                   
                       mean   min  
file_id                            
6142509188972423790    <4KB  <4KB  
7701937538982921416   ~64KB  <4KB  
7639450094153155784  ~256KB  <4KB  
7668160541777642696   ~16KB  <4KB  
7707004088563713224   ~16KB  <4KB  
...                     ...   ...  
8459486724935419126    <4KB  <4KB  
8459710870688662774    <4KB  <4KB  
1571755611344956586    <4KB  <4KB  
1571755611836627269    <4KB  <4KB  
1571755609301476523    <4KB  <4KB  

[1924 rows x 21 columns]

In [34]:
file_agg_sum = file_agg.groupby(level=0).sum().sort_values(('duration', 'sum'), ascending=False)
file_agg_sum


acc_pat         app        bw                    duration  \
                        max min nunique       per           sum           per   
file_id                                                                         
6142509188972423790       0   0       1  0.000000  0.000000e+00  5.626165e-01   
7701937538982921416       0   0       2  0.000283  7.137414e+08  4.874895e-02   
7639450094153155784       0   0       2  0.002021  5.093748e+09  2.303736e-02   
7668160541777642696       0   0       2  0.000299  7.529625e+08  1.874344e-02   
7707004088563713224       0   0       2  0.000296  7.456980e+08  1.547006e-02   
...                     ...  ..     ...       ...           ...           ...   
8459486724935419126       0   0       1  0.000000  0.000000e+00  1.082124e-07   
8459710870688662774       0   0       1  0.000000  0.000000e+00  1.082124e-07   
1571755611344956586       0   0       1  0.000000  0.000000e+00  1.082124e-07   
1571755611836627269       0   0       1  0.000000  0.000000e+00  1.082124e-07   
1571755609301476523       0   0       1  0.000000  0.000000e+00  1.082124e-07   

                               hostname   index               io_cat      \
                           sum  nunique   count           per    max min   
file_id                                                                    
6142509188972423790  45.752838       28     456  9.123963e-05      0   0   
7701937538982921416   3.964339        2  153464  3.070614e-02      3   1   
7639450094153155784   1.873434        2   55454  1.109562e-02      3   1   
7668160541777642696   1.524246        4  178024  3.562027e-02      3   1   
7707004088563713224   1.258049        2   66459  1.329758e-02      3   1   
...                        ...      ...     ...           ...    ...  ..   
8459486724935419126   0.000009        1       1  2.000869e-07      3   3   
8459710870688662774   0.000009        1       1  2.000869e-07      3   3   
1571755611344956586   0.000009        1       1  2.000869e-07      3   3   
1571755611836627269   0.000009        1       1  2.000869e-07      3   3   
1571755609301476523   0.000009        1       1  2.000869e-07      3   3   

                    proc_id    rank      size              
                    nunique nunique       per         sum  
file_id                                                    
6142509188972423790     114      13  0.000000           0  
7701937538982921416       5       5  0.074642  2829513124  
7639450094153155784       2       1  0.251737  9542800389  
7668160541777642696      10       5  0.030276  1147699816  
7707004088563713224       5       5  0.024748   938124444  
...                     ...     ...       ...         ...  
8459486724935419126       1       1  0.000000           0  
8459710870688662774       1       1  0.000000           0  
1571755611344956586       1       1  0.000000           0  
1571755611836627269       1       1  0.000000           0  
1571755609301476523       1       1  0.000000           0  

[1924 rows x 16 columns]

In [35]:
file_agg_sum_desc = file_agg_sum.describe(percentiles=[1-PERCENTILE,PERCENTILE])
file_agg_sum_desc

acc_pat                  app           bw                \
               max     min      nunique          per           sum   
count  1924.000000  1924.0  1924.000000  1924.000000  1.924000e+03   
mean      0.297297     0.0     1.846154     0.000520  1.310291e+09   
std       0.457187     0.0     0.537933     0.000448  1.129918e+09   
min       0.000000     0.0     1.000000     0.000000  0.000000e+00   
5%        0.000000     0.0     1.000000     0.000000  0.000000e+00   
50%       0.000000     0.0     2.000000     0.000361  9.102417e+08   
95%       1.000000     0.0     3.000000     0.001280  3.227667e+09   
max       1.000000     0.0     3.000000     0.002021  5.093748e+09   

           duration                  hostname          index                \
                per          sum      nunique          count           per   
count  1.924000e+03  1924.000000  1924.000000    1924.000000  1.924000e+03   
mean   5.197506e-04     0.042267     1.154366    2597.623701  5.197505e-04   
std    1.292229e-02     1.050861     1.138179   14402.934433  2.881839e-03   
min    1.082124e-07     0.000009     1.000000       1.000000  2.000869e-07   
5%     5.779526e-07     0.000047     1.000000       4.000000  8.003477e-07   
50%    5.418429e-05     0.004406     1.000000    1444.000000  2.889255e-04   
95%    2.395489e-04     0.019480     2.000000    2686.850000  5.376035e-04   
max    5.626165e-01    45.752838    31.000000  197957.000000  3.960861e-02   

            io_cat                   proc_id         rank         size  \
               max          min      nunique      nunique          per   
count  1924.000000  1924.000000  1924.000000  1924.000000  1924.000000   
mean      2.998441     1.281185     2.219335     1.340956     0.000520   
std       0.068394     0.654151     3.088186     1.104129     0.006373   
min       0.000000     0.000000     1.000000     1.000000     0.000000   
5%        3.000000     1.000000     1.000000     1.000000     0.000000   
50%       3.000000     1.000000     2.000000     1.000000     0.000112   
95%       3.000000     3.000000     5.000000     4.000000     0.000214   
max       3.000000     3.000000   114.000000    13.000000     0.251737   

                     
                sum  
count  1.924000e+03  
mean   1.970260e+07  
std    2.415860e+08  
min    0.000000e+00  
5%     0.000000e+00  
50%    4.253762e+06  
95%    8.094673e+06  
max    9.542800e+09

In [36]:
file_agg_cut = file_agg_sum[file_agg_sum[('duration', 'sum')] > file_agg_sum_desc.loc[PERCENTILE_FMT]['duration']['sum']].sort_values(('duration', 'sum'), ascending=False)
file_agg_cut

acc_pat         app        bw                duration  \
                        max min nunique       per           sum       per   
file_id                                                                     
6142509188972423790       0   0       1  0.000000  0.000000e+00  0.562616   
7701937538982921416       0   0       2  0.000283  7.137414e+08  0.048749   
7639450094153155784       0   0       2  0.002021  5.093748e+09  0.023037   
7668160541777642696       0   0       2  0.000299  7.529625e+08  0.018743   
7707004088563713224       0   0       2  0.000296  7.456980e+08  0.015470   
...                     ...  ..     ...       ...           ...       ...   
876519524468303606        0   0       2  0.000079  1.987389e+08  0.000258   
876519525291066038        0   0       3  0.000140  3.530464e+08  0.000256   
1236807497727822529       0   0       3  0.000130  3.272924e+08  0.000252   
7433760585119200025       0   0       2  0.000082  2.074422e+08  0.000248   
7433760585097911602       0   0       3  0.000169  4.252709e+08  0.000240   

                               hostname   index           io_cat     proc_id  \
                           sum  nunique   count       per    max min nunique   
file_id                                                                        
6142509188972423790  45.752838       28     456  0.000091      0   0     114   
7701937538982921416   3.964339        2  153464  0.030706      3   1       5   
7639450094153155784   1.873434        2   55454  0.011096      3   1       2   
7668160541777642696   1.524246        4  178024  0.035620      3   1      10   
7707004088563713224   1.258049        2   66459  0.013298      3   1       5   
...                        ...      ...     ...       ...    ...  ..     ...   
876519524468303606    0.021012        1    1461  0.000292      3   1       2   
876519525291066038    0.020843        2    2575  0.000515      3   1       4   
1236807497727822529   0.020476        2    2354  0.000471      3   1       6   
7433760585119200025   0.020145        1    1462  0.000293      3   1       2   
7433760585097911602   0.019483        2    2895  0.000579      3   1       5   

                       rank      size              
                    nunique       per         sum  
file_id                                            
6142509188972423790      13  0.000000           0  
7701937538982921416       5  0.074642  2829513124  
7639450094153155784       1  0.251737  9542800389  
7668160541777642696       5  0.030276  1147699816  
7707004088563713224       5  0.024748   938124444  
...                     ...       ...         ...  
876519524468303606        1  0.000110     4176002  
876519525291066038        3  0.000194     7358406  
1236807497727822529       5  0.000177     6701770  
7433760585119200025       1  0.000110     4178882  
7433760585097911602       4  0.000219     8285766  

[97 rows x 16 columns]

In [37]:
file_agg_cut_bw = file_agg_sum[file_agg_sum[('bw', 'sum')] < file_agg_sum_desc.loc['5%']['bw']['sum']].sort_values(('bw', 'sum'), ascending=True)
file_agg_cut_bw

Empty DataFrame
Columns: [(acc_pat, max), (acc_pat, min), (app, nunique), (bw, per), (bw, sum), (duration, per), (duration, sum), (hostname, nunique), (index, count), (index, per), (io_cat, max), (io_cat, min), (proc_id, nunique), (rank, nunique), (size, per), (size, sum)]
Index: []

In [38]:
%%time
proc_agg = ddf[ddf['io_cat'].isin([1, 2, 3])] \
    .groupby(['proc_id']) \
    .agg({
        'acc_pat': [min, max],
        'app': [nunique()],
        'duration': [sum],
        'file_id': [nunique()],
        'hostname': [nunique()],
        'index': ['count'],
        'io_cat': [min, max],
        # 'proc_id': [nunique()],
        'rank': [nunique()],
        'size': [min, max, 'mean', sum],
    }) \
    .compute()
proc_agg = proc_agg.sort_values(('duration', 'sum'), ascending=False)
proc_agg = format_agg(proc_agg)
proc_agg


CPU times: user 1 s, sys: 216 ms, total: 1.22 s
Wall time: 5.59 s


acc_pat         app        bw                          \
                        max min nunique       fmt       per           sum   
proc_id                                                                     
9060729382066249156       0   0       1  0.71GB/s  0.004725  7.639902e+08   
5901139297808997828       0   0       1  4.89GB/s  0.032458  5.248544e+09   
1029742319298993604       0   0       1  0.83GB/s  0.005491  8.879357e+08   
2463791410776694212       0   0       1  0.87GB/s  0.005759  9.313227e+08   
244024996488338884        0   0       1  2.07GB/s  0.013748  2.223145e+09   
...                     ...  ..     ...       ...       ...           ...   
6347012930982169908       1   0       1  0.00GB/s  0.000029  4.716893e+06   
1458149265372138804       1   0       1  0.00GB/s  0.000030  4.802287e+06   
6032102544405220660       1   0       1  0.00GB/s  0.000030  4.820660e+06   
1029740493937888564       1   0       1  0.00GB/s  0.000032  5.130035e+06   
2402798915213643060       1   0       1  0.01GB/s  0.000033  5.400482e+06   

                     duration           file_id hostname  ...     index  \
                          per       sum nunique  nunique  ...       per   
proc_id                                                   ...             
9060729382066249156  0.100803  3.585443       2        1  ...  0.029692   
5901139297808997828  0.050701  1.803385       2        1  ...  0.010988   
1029742319298993604  0.028866  1.026742       2        1  ...  0.012858   
2463791410776694212  0.027754  0.987185       2        1  ...  0.012742   
244024996488338884   0.026620  0.946837       2        1  ...  0.009717   
...                       ...       ...     ...      ...  ...       ...   
6347012930982169908  0.000012  0.000427       2        1  ...  0.000006   
1458149265372138804  0.000012  0.000420       2        1  ...  0.000006   
6032102544405220660  0.000012  0.000418       2        1  ...  0.000006   
1029740493937888564  0.000011  0.000392       2        1  ...  0.000006   
2402798915213643060  0.000010  0.000373       2        1  ...  0.000006   

                    io_cat        rank    size                            \
                       max min nunique     fmt           per         sum   
proc_id                                                                    
9060729382066249156      3   1       1  2.55GB  7.226069e-02  2739243777   
5901139297808997828      3   1       1  8.82GB  2.496886e-01  9465147906   
1029742319298993604      3   1       1  0.85GB  2.404996e-02   911681127   
2463791410776694212      3   1       1  0.86GB  2.425325e-02   919387406   
244024996488338884       3   1       1  1.96GB  5.552833e-02  2104957047   
...                    ...  ..     ...     ...           ...         ...   
6347012930982169908      3   2       1  0.00GB  5.318166e-08        2016   
1458149265372138804      3   2       1  0.00GB  5.318166e-08        2016   
6032102544405220660      3   2       1  0.00GB  5.318166e-08        2016   
1029740493937888564      3   2       1  0.00GB  5.307614e-08        2012   
2402798915213643060      3   2       1  0.00GB  5.318166e-08        2016   

                      xfer                
                       max    mean   min  
proc_id                                   
9060729382066249156   ~4MB   ~64KB  <4KB  
5901139297808997828   ~4MB  ~256KB  <4KB  
1029742319298993604  ~16MB   ~16KB  <4KB  
2463791410776694212   ~4MB   ~16KB  <4KB  
244024996488338884   ~16MB   ~64KB  <4KB  
...                    ...     ...   ...  
6347012930982169908   <4KB    <4KB  <4KB  
1458149265372138804   <4KB    <4KB  <4KB  
6032102544405220660   <4KB    <4KB  <4KB  
1029740493937888564   <4KB    <4KB  <4KB  
2402798915213643060   <4KB    <4KB  <4KB  

[255 rows x 21 columns]

In [40]:
proc_agg_desc = proc_agg.describe(percentiles=[1-PERCENTILE,PERCENTILE])
proc_agg_desc

acc_pat            app          bw                  duration  \
              max    min nunique         per           sum         per   
count  255.000000  255.0   255.0  255.000000  2.550000e+02  255.000000   
mean     0.780392    0.0     1.0    0.003922  6.341278e+08    0.003922   
std      0.414795    0.0     0.0    0.004477  7.239092e+08    0.008493   
min      0.000000    0.0     1.0    0.000003  4.324881e+05    0.000010   
5%       0.000000    0.0     1.0    0.000027  4.351799e+06    0.000013   
50%      1.000000    0.0     1.0    0.002116  3.421573e+08    0.001215   
95%      1.000000    0.0     1.0    0.013932  2.252885e+09    0.015307   
max      1.000000    0.0     1.0    0.032458  5.248544e+09    0.100803   

                      file_id hostname          index             io_cat  \
              sum     nunique  nunique          count         per    max   
count  255.000000  255.000000    255.0     255.000000  255.000000  255.0   
mean     0.139485   16.298039      1.0   19597.537255    0.003922    3.0   
std      0.302071   17.617921      0.0   38508.692940    0.007706    0.0   
min      0.000373    2.000000      1.0      28.000000    0.000006    3.0   
5%       0.000469    2.000000      1.0      28.000000    0.000006    3.0   
50%      0.043201    6.000000      1.0    1928.000000    0.000386    3.0   
95%      0.544436   53.000000      1.0  120762.800000    0.024165    3.0   
max      3.585443   53.000000      1.0  195672.000000    0.039155    3.0   

                     rank          size                
              min nunique           per           sum  
count  255.000000   255.0  2.550000e+02  2.550000e+02  
mean     1.101961     1.0  3.921569e-03  1.486580e+08  
std      0.303192     0.0  1.699033e-02  6.440659e+08  
min      1.000000     1.0  5.307614e-08  2.012000e+03  
5%       1.000000     1.0  5.318166e-08  2.016000e+03  
50%      1.000000     1.0  3.490250e-04  1.323077e+07  
95%      2.000000     1.0  1.379583e-02  5.229694e+08  
max      2.000000     1.0  2.496886e-01  9.465148e+09

In [41]:
proc_agg_cut_dur = proc_agg[proc_agg[('duration', 'sum')] > proc_agg_desc.loc[PERCENTILE_FMT]['duration']['sum']].sort_values(('duration', 'sum'), ascending=False)
proc_agg_cut_dur

acc_pat         app        bw                          \
                        max min nunique       fmt       per           sum   
proc_id                                                                     
9060729382066249156       0   0       1  0.71GB/s  0.004725  7.639902e+08   
5901139297808997828       0   0       1  4.89GB/s  0.032458  5.248544e+09   
1029742319298993604       0   0       1  0.83GB/s  0.005491  8.879357e+08   
2463791410776694212       0   0       1  0.87GB/s  0.005759  9.313227e+08   
244024996488338884        0   0       1  2.07GB/s  0.013748  2.223145e+09   
1701698302174617028       0   0       1  0.83GB/s  0.005505  8.902290e+08   
6032104343996521924       0   0       1  0.96GB/s  0.006356  1.027763e+09   
1705359014470147524       0   0       1  0.96GB/s  0.006395  1.034138e+09   
2439962764714761668       0   0       1  0.86GB/s  0.005690  9.200770e+08   
268174120124997060        0   0       1  0.88GB/s  0.005866  9.485460e+08   
6120541749300485572       0   0       1  0.56GB/s  0.003730  6.031823e+08   
226293052208111044        0   0       1  0.97GB/s  0.006425  1.038870e+09   
145425196050865604        0   0       1  0.77GB/s  0.005118  8.275500e+08   

                     duration           file_id hostname  ...     index  \
                          per       sum nunique  nunique  ...       per   
proc_id                                                   ...             
9060729382066249156  0.100803  3.585443       2        1  ...  0.029692   
5901139297808997828  0.050701  1.803385       2        1  ...  0.010988   
1029742319298993604  0.028866  1.026742       2        1  ...  0.012858   
2463791410776694212  0.027754  0.987185       2        1  ...  0.012742   
244024996488338884   0.026620  0.946837       2        1  ...  0.009717   
1701698302174617028  0.024081  0.856521       2        1  ...  0.010523   
6032104343996521924  0.020039  0.712768       2        1  ...  0.013324   
1705359014470147524  0.019989  0.710990       2        1  ...  0.013323   
2439962764714761668  0.019358  0.688530       2        1  ...  0.013342   
268174120124997060   0.016821  0.598314       2        1  ...  0.039155   
6120541749300485572  0.016588  0.590022       2        1  ...  0.024418   
226293052208111044   0.015646  0.556520       2        1  ...  0.012285   
145425196050865604   0.015325  0.545084       2        1  ...  0.031090   

                    io_cat        rank    size                         xfer  \
                       max min nunique     fmt       per         sum    max   
proc_id                                                                       
9060729382066249156      3   1       1  2.55GB  0.072261  2739243777   ~4MB   
5901139297808997828      3   1       1  8.82GB  0.249689  9465147906   ~4MB   
1029742319298993604      3   1       1  0.85GB  0.024050   911681127  ~16MB   
2463791410776694212      3   1       1  0.86GB  0.024253   919387406   ~4MB   
244024996488338884       3   1       1  1.96GB  0.055528  2104957047  ~16MB   
1701698302174617028      3   1       1  0.71GB  0.020115   762500039  ~16MB   
6032104343996521924      3   1       1  0.68GB  0.019325   732557229  ~16MB   
1705359014470147524      3   1       1  0.68GB  0.019396   735261354  ~16MB   
2439962764714761668      3   1       1  0.59GB  0.016712   633500885  ~16MB   
268174120124997060       3   1       1  0.53GB  0.014971   567528401   ~4MB   
6120541749300485572      3   1       1  0.33GB  0.009388   355890596  ~16MB   
226293052208111044       3   1       1  0.54GB  0.015252   578151227  ~16MB   
145425196050865604       3   1       1  0.42GB  0.011900   451084609   ~4MB   

                                   
                       mean   min  
proc_id                            
9060729382066249156   ~64KB  <4KB  
5901139297808997828  ~256KB  <4KB  
1029742319298993604   ~16KB  <4KB  
2463791410776694212   ~16KB  <4KB  
244024996488338884    ~64KB  <4KB  
1701698302174617028   ~16KB  <4KB

In [42]:
proc_agg_cut_bw = proc_agg[proc_agg[('bw', 'sum')] < proc_agg_desc.loc['5%']['bw']['sum']].sort_values(('bw', 'sum'), ascending=True)
proc_agg_cut_bw

acc_pat         app        bw                          \
                        max min nunique       fmt       per           sum   
proc_id                                                                     
115745060775054644        1   0       1  0.00GB/s  0.000003  4.324881e+05   
7510373950602399028       1   0       1  0.00GB/s  0.000011  1.803704e+06   
1701695905582861620       1   0       1  0.00GB/s  0.000014  2.229116e+06   
7825773834602073396       1   0       1  0.00GB/s  0.000014  2.240996e+06   
4459700650674015540       1   0       1  0.00GB/s  0.000015  2.407452e+06   
6082818377229847860       1   0       1  0.00GB/s  0.000015  2.423951e+06   
6439047109403528500       1   0       1  0.00GB/s  0.000021  3.384822e+06   
6361742405359684916       1   0       1  0.00GB/s  0.000021  3.442035e+06   
145422838113815860        1   0       1  0.00GB/s  0.000022  3.549296e+06   
244023136767495476        1   0       1  0.00GB/s  0.000022  3.630458e+06   
1134765578335079732       1   0       1  0.00GB/s  0.000024  3.851455e+06   
6120539923939380532       1   0       1  0.00GB/s  0.000026  4.251371e+06   
2439960978008362292       1   0       1  0.00GB/s  0.000026  4.253165e+06   

                     duration           file_id hostname  ...     index  \
                          per       sum nunique  nunique  ...       per   
proc_id                                                   ...             
115745060775054644   0.000131  0.004661       2        1  ...  0.000006   
7510373950602399028  0.000031  0.001118       2        1  ...  0.000006   
1701695905582861620  0.000025  0.000903       2        1  ...  0.000006   
7825773834602073396  0.000025  0.000900       2        1  ...  0.000006   
4459700650674015540  0.000024  0.000837       2        1  ...  0.000006   
6082818377229847860  0.000023  0.000832       2        1  ...  0.000006   
6439047109403528500  0.000017  0.000596       2        1  ...  0.000006   
6361742405359684916  0.000016  0.000586       2        1  ...  0.000006   
145422838113815860   0.000016  0.000568       2        1  ...  0.000006   
244023136767495476   0.000016  0.000554       2        1  ...  0.000006   
1134765578335079732  0.000015  0.000522       2        1  ...  0.000006   
6120539923939380532  0.000013  0.000474       2        1  ...  0.000006   
2439960978008362292  0.000013  0.000474       2        1  ...  0.000006   

                    io_cat        rank    size                      xfer  \
                       max min nunique     fmt           per   sum   max   
proc_id                                                                    
115745060775054644       3   2       1  0.00GB  5.318166e-08  2016  <4KB   
7510373950602399028      3   2       1  0.00GB  5.318166e-08  2016  <4KB   
1701695905582861620      3   2       1  0.00GB  5.307614e-08  2012  <4KB   
7825773834602073396      3   2       1  0.00GB  5.318166e-08  2016  <4KB   
4459700650674015540      3   2       1  0.00GB  5.318166e-08  2016  <4KB   
6082818377229847860      3   2       1  0.00GB  5.318166e-08  2016  <4KB   
6439047109403528500      3   2       1  0.00GB  5.318166e-08  2016  <4KB   
6361742405359684916      3   2       1  0.00GB  5.318166e-08  2016  <4KB   
145422838113815860       3   2       1  0.00GB  5.318166e-08  2016  <4KB   
244023136767495476       3   2       1  0.00GB  5.307614e-08  2012  <4KB   
1134765578335079732      3   2       1  0.00GB  5.307614e-08  2012  <4KB   
6120539923939380532      3   2       1  0.00GB  5.318166e-08  2016  <4KB   
2439960978008362292      3   2       1  0.00GB  5.318166e-08  2016  <4KB   

                                 
                     mean   min  
proc_id                          
115745060775054644   <4KB  <4KB  
7510373950602399028  <4KB  <4KB  
1701695905582861620  <4KB  <4KB  
7825773834602073396  <4KB  <4KB  
4459700650674015540  <4KB  <4KB  
6082818377229847860  <4KB  <4KB  
6439047109403528500  <4KB  <4KB  
6361742405359684916  <4KB  <4